<a href="https://colab.research.google.com/github/harrisontg57/NLP_Spring_2021/blob/main/Exam1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tim Harrison
Exam 1
Student ID: 002-24-5948

In [16]:
#Handling file imports here and creating dataframes
from os import walk
import numpy as np
import random
random.seed(12345)

_, _, filenames_neg = next(walk("TRAINING/negative"))
_, _, filenames_pos = next(walk("TRAINING/positive"))
#print(filenames_neg[0:10])
ones = [1] * len(filenames_pos)
zeroes = [0] * len(filenames_neg)
pos_txt = []
for name in filenames_pos:
  with open('/content/TRAINING/positive/'+name, 'r') as file:
      d = file.read().replace('\n', ' ')
      pos_txt.append(d)
df_pos = pd.DataFrame(data={'text': pos_txt, 'sentiment': ones})
neg_txt = []
for name in filenames_neg:
  with open('/content/TRAINING/negative/'+name, 'r') as file:
      d = file.read().replace('\n', ' ')
      neg_txt.append(d)
df_neg = pd.DataFrame(data={'text': neg_txt, 'sentiment': zeroes})

#df_neg.sample(frac=0.75,random_state=12345)

data = df_pos.append(df_neg)
#data.sample(frac=0.75,random_state=12345)

,text,sentiment
189,"Okay, now what the hell is this supposed to be...",0
1498,The only reason I wanted to see this was becau...,1
826,"There are few comedies like this, where almost...",1
850,This is a very good movie. Do you want to know...,1
748,"""Margaritas and Cock...""<br /><br />This treme...",1
...,...,...
1527,My choice for greatest movie ever used to be L...,1
1370,"The Haunted World of Edward D. Wood, Jr. isn't...",1
515,Perhaps the funniest 'backstage at Hollywood' ...,1
883,This is a powerful documentary about domestic ...,1


Question 1)

In [18]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline, FeatureUnion

def create_model(alg_name, vectorization, train):
  if alg_name == "naive_bayes":
    learn = MultinomialNB()
  elif alg_name == "perceptron":
    learn = Perceptron()
  elif alg_name == "logistic_regression":
    learn = LogisticRegression()
  else:
    raise NameError('incorrect algorithm name')

  if vectorization == "tfidf":
    vect = TfidfVectorizer()
  elif vectorization == "bag_of_words":
    vect = CountVectorizer()
  elif vectorization == "both":
    vect = FeatureUnion([("tfidf", TfidfVectorizer()), ("bag_of_words", CountVectorizer())])
  else:
    raise NameError('incorrect vectorization name')
  
  model = make_pipeline(vect, learn)
  model.fit(train.text, train.sentiment)
  return model


Question 2)

In [21]:
from sklearn.model_selection import train_test_split
train_a, test_a = train_test_split(data, test_size=0.25,random_state=12345)
model_a = create_model("naive_bayes","both",train_a)


Pipeline(memory=None,
         steps=[('featureunion',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('tfidf',
                                                 TfidfVectorizer(analyzer='word',
                                                                 binary=False,
                                                                 decode_error='strict',
                                                                 dtype=<class 'numpy.float64'>,
                                                                 encoding='utf-8',
                                                                 input='content',
                                                                 lowercase=True,
                                                                 max_df=1.0,
                                                                 max_features=None,
                                                                 min_df=1,
                 

Question 3)

In [24]:
labels = model_a.predict(test_a.text)
pd.DataFrame(data={'text': test_a.text, 'true_sentiment': test_a.sentiment, 'Predicted_sentiment': labels})

,text,true_sentiment,Predicted_sentiment
189,"Okay, now what the hell is this supposed to be...",0,0
1498,The only reason I wanted to see this was becau...,1,0
826,"There are few comedies like this, where almost...",1,1
850,This is a very good movie. Do you want to know...,1,1
748,"""Margaritas and Cock...""<br /><br />This treme...",1,1
...,...,...,...
1313,I saw this movie when it first came out. It wa...,1,1
381,I absolutely loved this movie. I am not even s...,1,1
1172,"PERHAPS in an attempt to find another ""Hot Pro...",1,1
708,I cannot believe this woodenly written and dir...,0,0
